### One Class Support Vector Machine

公式ドキュメント：https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html<br>

まず，OneClassSupportVectorMachineは異常検知の手法のうち教師データなしのものである．<br>
教師なし学習とは,データそのものが持つ構造を浮かび上がらせることで,似たデータ同士にグループ化して分類する手法である．<br>

詳細については以下のリンクをみてみるといいかもしれない<br>
[異常検知のための One Class SVM](https://qiita.com/kznx/items/434d98bf1a0e39327542)<br>
[One Class SVMを用いた異常検知](https://www.slideshare.net/YutoMori2/one-class-svm)<br>
[One Class Support Vector Machine(One Class SVM入門)](https://recruit.cct-inc.co.jp/tecblog/machine-learning/one-class-svm/)

#### はじめに

wineデータセットに対してOneClassSupportVectorMachineを使用する<br>
スケーリング:標準化(平均0, 分散１)<br>
今回は学習用データとテスト用データは以下の通りにする<br>
学習用：正常データ38個<br>
テスト用：正常データ10個, 異常データ10個<br>

目的<br>
正常データをclass_0としてOCSVMで学習し, FARとFRRで評価を行う．<br>

version.など<br>
python 3.7.7<br>
scikit-learn==0.23.2<br>
pandas==1.1.5<br>

In [46]:
# 必要なパッケージを用意
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

# スケーリング
from sklearn import preprocessing

# モデル
from sklearn.svm import OneClassSVM

# warning ignore code
import warnings
warnings.filterwarnings('ignore')

# 表示範囲設定
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

In [47]:
# データの用意: 詳細についてはdataset.ipynbを参照
from sklearn.datasets import load_wine
data_wine = load_wine()

df = pd.DataFrame(data_wine["data"],columns=data_wine["feature_names"])
df['target'] = data_wine['target']
df['target'] = df['target'].replace({0:'class_0', 1:'class_1', 2:'class_2'})

df_all = df.copy()

In [48]:
# target_nameに指定したラベルを正常データ
# それ以外を異常データとして分ける
# 今回はclass_2にあわせて, 学習用に正常データ38個, テスト用に正常データ10個と異常データ10個を使用する

def select_data(df_target, target_name):
    # target_nameで指定したものをdf1
    df1 = df_target[df_target['target'] == target_name]
    # target_nameで指定したもの以外をdf2
    df2 = df_target[df_target['target'] != target_name]
    
    # 正常データ(本人)
    x_normal = df1.drop('target', axis=1)
    y_normal = df1['target']
    
    # 異常データ(他人)
    x_anomaly = df2.drop('target', axis=1)
    y_anomaly = df2['target']
    
    # 
    # 今回は学習用データの数をそろえる為, train_seizeはclass_2の数からテスト用のデータ数１０個を引いたもの
    # train_size, test_seize:0.0~1.0の間で割合 or 個数
    x_train_no, x_test_no, y_train_no, y_test_no = train_test_split(x_normal, y_normal, train_size=38, test_size=10, random_state=0, shuffle=True)
    _x_train_ano, x_test_ano, _y_train_ano, y_test_ano = train_test_split(x_anomaly, y_anomaly, test_size=10, random_state=0, shuffle=True)
    
    return x_train_no, x_test_no, y_train_no, y_test_no, x_test_ano, y_test_ano

In [49]:
# class_0の部分はclass_1やclass_2でもOK
X_train_no, X_test_no, Y_train_no, Y_test_no, X_test_ano, Y_test_ano = select_data(df_all, 'class_0')

In [ ]:
# 各データの確認

In [ ]:
X_train_no.head()

In [ ]:
X_test_no

In [ ]:
Y_train_no

In [ ]:
Y_test_no

In [ ]:
X_test_ano

In [ ]:
Y_test_ano

#### 前処理：標準化（平均０, 分散１）

In [ ]:
ss = preprocessing.StandardScaler()
ss.fit(X_train_no)
x_train_ss = ss.transform(X_train_no)
x_test_no_ss = ss.transform(X_test_no)
x_test_ano_ss = ss.transform(X_test_ano)

#### ここから学習していく

In [54]:
# モデル作成
clf = OneClassSVM(nu=0.1, kernel="rbf")
# 学習
clf.fit(x_train_ss)
# 予測
pred_train = clf.predict(x_train_ss)
pred_test_no = clf.predict(x_test_no_ss)
pred_test_ano = clf.predict(x_test_ano_ss)

正常データは１， 異常データは-１として返される

In [55]:
pred_train

array([-1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1,  1, -1,
        1,  1,  1,  1, -1, -1,  1, -1,  1, -1,  1, -1,  1, -1, -1,  1,  1,
       -1, -1,  1, -1])

In [ ]:
# 細かい数値がほしいならこちら： 上記の結果と比較するとわかるが0より大きい場合は正常，小さい場合は異常と判断される
clf.decision_function(x_train_ss)

In [56]:
pred_test_no

array([ 1,  1, -1,  1, -1, -1,  1, -1,  1,  1])

In [ ]:
clf.decision_function(x_test_no_ss)

In [57]:
pred_test_ano

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [ ]:
clf.decision_function(x_test_ano_ss)

In [ ]:
Y_test_ano

In [50]:
# モデル作成
clf2 = OneClassSVM(nu=0.1, kernel="rbf")
# 学習
clf2.fit(x_train_ss)
# 予測
pred_train2 = clf.predict(X_train_no)
pred_test_no2 = clf.predict(X_test_no)
pred_test_no2  = clf.predict(X_test_ano)

In [51]:
pred_train2

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1])

In [52]:
pred_test_no2 

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [53]:
pred_test_no2 

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

#### 評価について

[混同行列(Confusion Matrix) とは 〜 2 値分類の機械学習のクラス分類について](https://qiita.com/TsutomuNakamura/items/a1a6a02cb9bb0dcbb37f)<br>

|  | 予測(Positive) | 予測（Negative） |
| :--- | :---: | :---: |
| 実際(Positive) | TP(True Positive) | FN(False Negative) |
| 実際(Negative) | FP(False Positive) | TN(True Negative) |<br>
<br>

・真陽性（TP: True Positive）: 実際のクラスがPositiveで予測もPositive（正解）<br>
・真陰性（TN: True Negative）: 実際のクラスがNegativeで予測もNegative（正解）<br>
・偽陽性（FP: False Positive）: 実際のクラスはNegativeで予測がPositive（不正解）<br>
・偽陰性（FN: False Negative）: 実際のクラスはPositiveで予測がNegative（不正解）<br>

他人受入率;誤受理率（False Acceptance Rate; FAR） ＝ FP / (TN + FP) =  FPR<br>
他人と判断すべきなのに本人と判断した<br>

本人拒否率;誤棄却率（False Rejection Rate; FRR） ＝ FN / (FN + TP) = FNR<br>
本人と判断すべきなのに他人と判断した<br>

In [ ]:
# 評価用関数
def far_frr(normal_result, anomaly_result):
    tp = np.count_nonzero(normal_result == 1)
    fn = np.count_nonzero(normal_result == -1)
    fp = np.count_nonzero(anomaly_result == 1)
    tn = np.count_nonzero(anomaly_result == -1)
    re_accuracy = (tp + tn) / (tp + fn + fp + tn)
    re_far = fp / (tn + fp)
    re_frr = fn / (fn + tp)

    # accuracy = ((TP+TN)/(TP+FN+FP+TN))
    # print(accuracy)
    return re_far, re_frr, re_accuracy

In [ ]:
far, frr, accuracy= far_frr(pred_test_no, pred_test_ano)

In [ ]:
far

In [ ]:
frr

In [ ]:
accuracy

In [ ]:
# 異常データはきちんと弾いたが一部正常データも弾いた

In [35]:
from tqdm import tqdm

In [ ]:
# ハイパーパラメータいじるならこんな感じ

In [36]:
from sklearn.metrics import accuracy_score
def test(nu, gamma):
    # モデル作成
    clf = OneClassSVM(nu=nu, kernel="rbf", gamma=gamma)
    # 学習
    clf.fit(x_train_ss)
    # 予測
    pred_train = clf.predict(x_train_ss)
    pred_test_no = clf.predict(x_test_no_ss)
    pred_test_ano = clf.predict(x_test_ano_ss)

    far, frr, accuracy = far_frr(pred_test_no, pred_test_ano)
#     print(f'contamination={nu} and gamma={gamma}')
#     print(f'FAR:{far}\nFRR{frr}\nAccuracy{accuracy}\n')
    return far, frr, accuracy
    

In [37]:
contamination=0.1
lst=[]
for nu in tqdm(np.linspace(0.01, 1, 100)):
    for gamma in np.linspace(0.01, 1, 50):
        far, frr, accuracy = test(nu, gamma)
        lst.append([nu, gamma, far, frr, accuracy])

100%|██████████| 100/100 [00:03<00:00, 33.08it/s]


In [39]:
result = pd.DataFrame(lst)

In [40]:
result.columns = ['nu', 'gamma', 'far', 'frr', 'accuracy']

In [45]:
result.to_csv('result.csv')